In [1]:
import os
import re

In [2]:
def metamap_recommend(term, dictionary, topk=10):
    semantic_type = 'acab,anab,comd,cgab,dsyn,emod,fndg,inpo,mobd,neop,patf,sosy'
    if dictionary == 'disorder_dictionary':
        var = os.popen("echo " + term + " | " + f'./public_mm/bin/metamap -I -y -Z 2020AB -V Base -c -b -y -J {semantic_type}').read()
    elif dictionary == 'icd10_dictionary':
        var = os.popen("echo " + term + " | " + f'./public_mm/bin/metamap -I -y -Z 2020AB -V Base -c -b -y -R ICD10').read()
    else:
        var = os.popen("echo " + term + " | " + f'./public_mm/bin/metamap -I -y -Z 2020AB -V Base -c -b -y -R HPO').read()

    cuis = var.split('\n')
    candidates_cui = []
    scores = []
    for item in cuis[4:]:
        cui = re.search(r'(\d+).*(C\d+)', item)
        if cui != None:
            cui_id = cui.group(2)
            score = cui.group(1).strip(' ')
            candidates_cui.append(cui_id)
            scores.append(float(score))
    if len(scores) < 10:
        scores.extend([0 for i in range(0,10-len(scores))])
        candidates_cui.extend(['null' for i in range(0,10-len(candidates_cui))])
    return candidates_cui[:topk], scores[:topk]

In [3]:
candidates , scores = metamap_recommend('kidney','disorder_dictionary')
print("start")
print(scores)
print(candidates)

start
[1000.0, 1000.0, 907.0, 907.0, 0, 0, 0, 0, 0, 0]
['C0022646', 'C0227665', 'C0027697', 'C0027697', 'null', 'null', 'null', 'null', 'null', 'null']


In [4]:
import numpy as np 
np.stack(np.array([np.array(scores),np.array(scores)]),axis=0)

array([[1000., 1000.,  907.,  907.,    0.,    0.,    0.,    0.,    0.,
           0.],
       [1000., 1000.,  907.,  907.,    0.,    0.,    0.,    0.,    0.,
           0.]])